In [3]:
import torch
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import torch.nn.functional as F

from torch import nn
from torchvision import datasets, transforms
from PIL import Image

from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.handlers import EarlyStopping

In [4]:
# bez tego wysadza kernel kiedy rysuje obrazek
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [43]:
classes = {
    0: "plane",
    1: "car",
    2: "bird",
    3: "cat",
    4: "deer",
    5: "dog",
    6: "frog",
    7: "horse",
    8: "ship",
    9: "truck"
}

device = "cuda"
batch_size = 64
torch.manual_seed(0)

In [6]:
transform = transforms.Compose([
    transforms.ToTensor()  # this includes scaling to [0, 1]
])

train_data = datasets.CIFAR10("./cifar10", download=True, transform=transform, train=True)
test_data = datasets.CIFAR10("./cifar10", download=True, transform=transform, train=False)

train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_data, shuffle=True, batch_size=batch_size)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
print(train_data)
print(test_data)
print(train_data[0][0].size())
print(train_data[0][0])

In [7]:
def print_img(img, axs):
    img_np = img.numpy()
    img_denormalized = (img_np*255).astype("uint8").transpose(1, 2, 0) 
    return axs.imshow(img_denormalized)

In [ ]:
fig, axs = plt.subplots(3, 5, constrained_layout=True)
axs = np.reshape(axs, -1)
for x in range(15):
    img, label = train_data[x]
    axs[x].title.set_text(classes[label])
    print_img(img, axs[x])

In [25]:
def run_model(model):
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    trainer = create_supervised_trainer(model, optimizer, loss_fn, device)

    val_metrics = {
        "accuracy": Accuracy(),
        "loss": Loss(loss_fn)
    }

    train_evaluator = create_supervised_evaluator(model, metrics=val_metrics, device=device)
    val_evaluator = create_supervised_evaluator(model, metrics=val_metrics, device=device)

    @trainer.on(Events.EPOCH_COMPLETED)
    def log_training_results(trainer):
        train_evaluator.run(train_loader)
        metrics = train_evaluator.state.metrics
        print(f"Training Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy'] * 100:.2f}%, Avg loss: {metrics['loss']:.2f}")


    @trainer.on(Events.EPOCH_COMPLETED)
    def log_validation_results(trainer):
        val_evaluator.run(test_loader)
        metrics = val_evaluator.state.metrics
        print(f"Validation Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy'] * 100:.2f}%, Avg loss: {metrics['loss']:.2f}")

    def score_function(engine):
        metrics = engine.state.metrics
        return metrics["accuracy"]
    
    val_evaluator.add_event_handler(Events.COMPLETED, EarlyStopping(3, score_function, trainer))

    trainer.run(train_loader, max_epochs=30)

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3*32*32, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
run_model(model)

In [44]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.conv3 = nn.Conv2d(16, 32, 5)
        self.conv4 = nn.Conv2d(32, 48, 5)
        # self.pool = nn.MaxPool2d(2, 2)
        # self.fc1 = nn.Linear(16 * 5 * 5, 120)
        # self.fc1 = nn.Linear(6 * 28 * 28, 120)
        self.fc1 = nn.Linear(32 * 16 * 16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 64)
        self.fc4 = nn.Linear(64, 10)
        self.fc5 = nn.Linear(32, 10)
        self.fc6 = nn.Linear(16, 10)

    def forward(self, x):
        # x = self.pool(F.relu(self.conv1(x)))
        # x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x
    
model = Net().to(device)
run_model(model)

Training Results - Epoch[1] Avg accuracy: 44.37%, Avg loss: 1.50
Validation Results - Epoch[1] Avg accuracy: 43.53%, Avg loss: 1.52
Training Results - Epoch[2] Avg accuracy: 55.74%, Avg loss: 1.24
Validation Results - Epoch[2] Avg accuracy: 53.38%, Avg loss: 1.30
Training Results - Epoch[3] Avg accuracy: 60.24%, Avg loss: 1.11
Validation Results - Epoch[3] Avg accuracy: 56.67%, Avg loss: 1.21
Training Results - Epoch[4] Avg accuracy: 64.18%, Avg loss: 1.01
Validation Results - Epoch[4] Avg accuracy: 59.00%, Avg loss: 1.15
Training Results - Epoch[5] Avg accuracy: 67.63%, Avg loss: 0.92
Validation Results - Epoch[5] Avg accuracy: 60.31%, Avg loss: 1.13
Training Results - Epoch[6] Avg accuracy: 70.12%, Avg loss: 0.85
Validation Results - Epoch[6] Avg accuracy: 61.41%, Avg loss: 1.14
Training Results - Epoch[7] Avg accuracy: 74.10%, Avg loss: 0.74
Validation Results - Epoch[7] Avg accuracy: 62.07%, Avg loss: 1.11
Training Results - Epoch[8] Avg accuracy: 74.74%, Avg loss: 0.72
Validation 

2023-03-24 23:08:10,734 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training


Validation Results - Epoch[12] Avg accuracy: 61.44%, Avg loss: 1.42


1 + 3 - Validation Results - Epoch[11] Avg accuracy: 55.59%, Avg loss: 1.34
2 + 3 - Validation Results - Epoch[10] Avg accuracy: 58.85%, Avg loss: 1.34

3 + 3 - Validation Results - Epoch[10] Avg accuracy: 58.77%, Avg loss: 1.23
3 + 4 - Validation Results - Epoch[12] Avg accuracy: 61.44%, Avg loss: 1.42
3 + 5 - Validation Results - Epoch[15] Avg accuracy: 57.74%, Avg loss: 1.47

4 + 3 - Validation Results - Epoch[11] Avg accuracy: 59.61%, Avg loss: 1.36
4 + 4 - Validation Results - Epoch[15] Avg accuracy: 61.82%, Avg loss: 1.52
4 + 5 - Validation Results - Epoch[19] Avg accuracy: 60.91%, Avg loss: 1.24

5 + 3 - Validation Results - Epoch[18] Avg accuracy: 60.97%, Avg loss: 1.17
5 + 4 - Validation Results - Epoch[14] Avg accuracy: 60.22%, Avg loss: 1.24
5 + 5 - Validation Results - Epoch[23] Avg accuracy: 61.34%, Avg loss: 1.33
5 + 6 - Validation Results - Epoch[20] Avg accuracy: 59.16%, Avg loss: 1.23

6 + 3 - Validation Results - Epoch[17] Avg accuracy: 60.48%, Avg loss: 1.22
